In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir("/Users/admin/Downloads/EBAC/dataframesEbac")
df = pd.read_csv("Advertising.csv")
df.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9


In [2]:
df["Intercepto"] = 1
df = df[['Intercepto', 'TV', 'Radio', 'Newspaper', 'Sales']]

Xdata = df[['Intercepto', 'TV', 'Radio', 'Newspaper']].values
Ydata = df[["Sales"]].values

In [3]:
# Bases de entrenamiento y de prueba
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Xdata, Ydata, test_size = 0.3, random_state = 1) 

In [4]:
x = x_train
y = y_train

In [5]:
# Formato numerico
np.set_printoptions(formatter = {'float_kind':'{:f}'.format})

In [6]:
XT_X = np.matmul(np.matrix.transpose(x), x)
XT_X_inv = np.linalg.inv(XT_X)
XT_Y = np.matmul(np.matrix.transpose(x), y)
betas = np.matmul(XT_X_inv, XT_Y)
betas

array([[4.661440],
       [0.055024],
       [0.102510],
       [-0.001507]])

In [7]:
# Calculo de los pronosticos para Y de acuerdo a los  coeficientes de regresion
Y_pred = np.matmul(x, betas)
# Calculo de residuales
Resid = y - Y_pred

In [8]:
# Calculo de la suma de residuales al cuadrado
RSS = float(np.matmul(np.matrix.transpose(Resid), Resid))
# Calculo de la suma total de cuadrados
TSS = float( np.matmul(np.matrix.transpose(y), y) - len(y) * (y.mean()**2))
# Calculo del coeficiente de determinación 
R_cuad = float(1 - RSS/TSS)
R_cuad

0.8993745840124557

In [9]:
# Calculo de la varianza del error de regresion
s_cuad = RSS / (len(y) - x.shape[1])
s_cuad

2.950210939339458

In [10]:
# Desviacion estandar del error de regresion
import math
s = math.sqrt(s_cuad)
s

1.7176178094498957

In [11]:
# Obtencion del valor critico de la t de student

import scipy.stats

# Grados de libertad n - (k+1)
grados_libertad = len(y) - x.shape[1]
confianza = 0.90
q = (1 - confianza) / 2
t_critico = abs(scipy.stats.t.ppf(q, df = grados_libertad))
t_critico

1.6561349882437968

In [12]:
# Valores particulares de x TV, Radio y Periodicos
f = np.array([[1], [100], [50], [70]])
f

array([[  1],
       [100],
       [ 50],
       [ 70]])

In [13]:
margen_error = t_critico * (s * (float(np.matmul(np.matmul(np.matrix.transpose(f), XT_X_inv), f)) ** 0.5))
margen_error

0.5917297446911736

In [14]:
pron_puntual  = float(np.matmul(np.matrix.transpose(f), betas))
pron_puntual

15.18383546573101

In [15]:
# limites del intervalo de confianza 
lim_inferior = pron_puntual - margen_error
lim_superior = pron_puntual + margen_error
print("Intervalo de confianza: (", lim_inferior, ',', lim_superior, ')')

Intervalo de confianza: ( 14.592105721039836 , 15.775565210422183 )


# Validacion de supuestos de la regresión:

## Supuesto 1 

In [18]:
import scipy
# Calculo de la asimetria en residuales
skew = float(scipy.stats.skew(Resid, bias = True))
skew

-0.5262788252488116

In [19]:
# Calculo de la curtosis de residuales
kurtosis = float(scipy.stats.kurtosis(Resid, fisher = False))
kurtosis

4.824267653812199

In [20]:
Jarque_bera = (len(x) / 6) * (skew ** 2 + (kurtosis - 3) ** 2 / 4)
Jarque_bera

25.875675468804822

In [21]:
nivel_confianza = 0.90
scipy.stats.chi2.ppf(nivel_confianza, df = 2)

4.605170185988092

Conclusión: Dado que JB es mayor al nivel critico, podemos rechazar la hipotesis de existencia de normalidad
en los residuales

## Supuesto 2 : Inexistencia de autocorrelacion entre residules

In [24]:
from statsmodels.formula.api import ols
y_df = pd.DataFrame(y)
x_df = pd.DataFrame(x[:, 1:])

In [25]:
df = pd.concat([y_df, x_df.reindex(y_df.index)], axis = 1)
df.columns = ['Y', 'X1', 'X2', 'X3']

In [26]:
# Ajuste de regresion lineal multiple 
model = ols('Y ~ X3', data = df).fit()

from statsmodels.stats.stattools import durbin_watson

# Prueba de Durbin-Watson
durbin_watson(model.resid)

2.232296893827158

Conclusion: Dado que DW si es aprox. igual a 2, no podemos pensar que existe autocorrelacion entre los residuales

## Supuesto 3: Homoceasticidad (igual de varianzas)

In [29]:
ResidCuad = Resid ** 2
ResidCuad = pd.DataFrame(ResidCuad)

In [30]:
X1 = df.iloc[:, 1:]
X1_df = pd.DataFrame(X1)
X1Cuad = X1 ** 2
X1Cuad_df = pd.DataFrame(X1Cuad)

In [31]:
df_Aux = pd.concat([ResidCuad, X1_df.reindex(y_df.index), X1Cuad_df.reindex(y_df.index)], axis = 1)
df_Aux.columns = ['Residual', 'X1', 'X2', 'X3', 'X1Cuad', 'X2Cuad', 'X3Cuad']

In [32]:
# Ajuste de regresión lineal múltiple 
modelAux = ols('Residual ~ X3 + X1Cuad + X2Cuad + X3Cuad', data = df_Aux).fit()
RSqAux = modelAux.rsquared
RSqAux

0.02861700968681591

In [33]:
Estadistico = len(y) * RSqAux
Estadistico

4.006381356154227

In [34]:
Nivel_confianza = 0.90
scipy.stats.chi2.ppf(Nivel_confianza, df = 2)

4.605170185988092

Conclusión : Al no superar al valor critico nuestro estadistico de prueba, no hay evidencia de Heterocedasticidad (desigualdad de varianza de los residuales)

In [36]:
# Alternativa para la prueba de white
from statsmodels.stats.diagnostic import  het_white

white_test = het_white(model.resid, model.model.exog)
print("Estadistico de prueba: ", white_test[0])
print("Valor p: ", white_test[1])

Estadistico de prueba:  5.021565693964494
Valor p:  0.0812046435411481


Conclusion: A un nivel de Alpha = 10%, como tenemos un valor p inferior a Alpha, podemos rechazar la hipotesis de Homocedasticidad (lo cual implica que existe evidencia de Homocedasticidad)